# Data Conversion of HydroCAT .hex data
This python notebook shows how to convert a .hex file into pandas dataframe with scientific values.

Please contact [SBS customer support](https://www.seabird.com/support) for help or to request additional features.

## Init

The initialization code section below is used to import required libraries.

In [6]:
# Native imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import gsw

# Sea-Bird imports
import seabirdscientific.conversion as conv
import seabirdscientific.instrument_data as id
import seabirdscientific.visualization as viz
import example_data.example_coefficients as ec

## [Data Conversion](#proc-list)

This section shows how to convert raw data contained in a .hex file into scientific units for the instruments that follow:  
- HydroCAT
- HydroCAT-ODO

In [2]:
hex_file = os.path.join("example_data", "HydroCAT", "HydroCAT-ODO.hex")

# Convert raw hexadecimal string to raw frequencies
raw_data = id.read_hex_file(
    filepath=hex_file,
    instrument_type=id.InstrumentType.HydroCATODO,
    enabled_sensors=[
        id.Sensors.Temperature,
        id.Sensors.Conductivity,
        id.Sensors.Pressure,
        id.Sensors.SBE63
    ],
)

In [3]:
raw_data

,temperature,conductivity,SBE63 oxygen phase,SBE63 oxygen temperature,pressure,temperature compensation,date time
0,233108.0,2622.671875,16.980,0.597280,553866.0,2203.0,2025-09-19 20:10:03
1,233084.0,2622.667969,16.981,0.597273,553868.0,2203.0,2025-09-19 20:10:13
2,233066.0,2622.667969,16.977,0.597262,553867.0,2203.0,2025-09-19 20:10:23
3,233048.0,2622.667969,16.982,0.597262,553867.0,2204.0,2025-09-19 20:10:33
4,233027.0,2622.664062,16.976,0.597249,553865.0,2203.0,2025-09-19 20:10:43
5,233003.0,2622.664062,16.979,0.597238,553867.0,2203.0,2025-09-19 20:10:53
6,232983.0,2622.667969,16.981,0.597230,553866.0,2204.0,2025-09-19 20:11:03
7,232961.0,2622.664062,16.981,0.597226,553867.0,2204.0,2025-09-19 20:11:13
8,232928.0,2622.664062,16.978,0.597218,553867.0,2203.0,2025-09-19 20:11:23
9,232909.0,2622.664062,16.978,0.597207,553867.0,2203.0,2025-09-19 20:11:33


In [10]:
# Convert raw frequencies to scientific values

temperature = conv.convert_temperature(
    temperature_counts_in=raw_data["temperature"].values,
    coefs=ec.temperature_coefs_sn03906502,
    standard='ITS90',
    units='C',
    use_mv_r=False,
)

pressure_dbar = conv.convert_pressure(
    pressure_count=raw_data["pressure"].values,
    compensation_voltage=raw_data["temperature compensation"],
    coefs=ec.pressure_coefs_sn03906502,
    units='dbar',
) 

conductivity = conv.convert_conductivity(
    conductivity_count=raw_data["conductivity"].values,
    temperature=temperature,
    pressure=pressure_dbar,  # must be in units of dbar
    coefs=ec.conductivity_coefs_sn6385
)

# salinity = gsw.SP_from_C(
#     C=conductivity / 1000, # mS/cm
#     t=temperature, # ITS90 C
#     p=pressure_dbar # dbar
# )
salinity = np.full(len(temperature), 30.0) # override until we have a real data set

oxygen = conv.convert_sbe63_oxygen(
    raw_oxygen_phase=raw_data["SBE63 oxygen phase"],
    thermistor=raw_data["SBE63 oxygen temperature"],
    pressure=pressure_dbar,
    salinity=salinity,
    coefs=ec.oxygen_63_coefs_sn2568,
    thermistor_coefs=ec.thermistor_63_coefs_sn2568,
    thermistor_units="volts"
)

# Flag to be used in data processing
flag = np.zeros(len(temperature))

data = pd.DataFrame({
    "temp": temperature,
    "cond":conductivity,
    "sal":salinity,
    "oxy":oxygen,
    "press":pressure_dbar,
    "date_time":raw_data["date time"],
    "flag":flag
})

data

,temp,cond,sal,oxy,press,date_time,flag
0,84.650202,-0.132421,30.0,5.172392,1130.969231,2025-09-19 20:10:03,0.0
1,84.653902,-0.132424,30.0,5.171595,1130.970529,2025-09-19 20:10:13,0.0
2,84.656677,-0.132424,30.0,5.174447,1130.969880,2025-09-19 20:10:23,0.0
3,84.659453,-0.132424,30.0,5.170763,1130.968627,2025-09-19 20:10:33,0.0
4,84.662691,-0.132426,30.0,5.175072,1130.968582,2025-09-19 20:10:43,0.0
5,84.666393,-0.132426,30.0,5.172766,1130.969880,2025-09-19 20:10:53,0.0
6,84.669478,-0.132424,30.0,5.171223,1130.967978,2025-09-19 20:11:03,0.0
7,84.672872,-0.132426,30.0,5.171188,1130.968627,2025-09-19 20:11:13,0.0
8,84.677963,-0.132426,30.0,5.173330,1130.969880,2025-09-19 20:11:23,0.0
9,84.680895,-0.132426,30.0,5.173234,1130.969880,2025-09-19 20:11:33,0.0


## [Data Plotting](#proc-list)

In [12]:
config = viz.ChartConfig(
    title="HydroCAT-ODO Data Conversion",
    x_names=["date_time"],
    y_names=["temp", "cond", "press", "oxy"],
    z_names=[],
    chart_type="overlay",
    plot_loop_edit_flags=False,
    lift_pen_over_bad_data=True,
)

chart_data = viz.ChartData(data, config)
fig = viz.plot_xy_chart(chart_data, config)

# plotly customizations
fig.data[0].name = "Temperature"
fig.data[1].name = "Conductivity"
fig.data[2].name = "Pressure"
fig.data[3].name = "Oxygen"
fig["layout"]["yaxis"]["title"] = "Temperature [ITS-90 degrees C]"
fig["layout"]["yaxis2"]["title"] = "Conductivity [S/m]"
fig["layout"]["yaxis3"]["title"] = "Pressure [dbar]"
fig["layout"]["yaxis4"]["title"] = "Oxygen [ml/l]"


fig.update_layout(height=800)
fig.show()